### Post Import Fixes
> Post fixes that need to be applied after importing litesearch package.

We need to apply some post fix for mac to load the dylib properly for usearch sqlite path to be loaded correctly.


In [ ]:
#| default_exp postfix

In [ ]:
#| export
import os, subprocess, platform

In [ ]:
#| export
_usearch_fix_applied = False

def usearch_fix():
    """Apply usearch macOS fix if on macOS. Safe to call multiple times."""
    global _usearch_fix_applied
    if _usearch_fix_applied:
        return  # Already applied
    
    print('Applying usearch macOS fix if required...')
    try:
        from usearch import sqlite_path
        dylib_path = sqlite_path()+'.dylib'
        print('usearch dylib path: ', dylib_path)
        if platform.system() != "Darwin": 
            print('Not on macOS, skipping usearch fix.')
            _usearch_fix_applied = True
            return
        cmd = ['install_name_tool', '-add_rpath', '/usr/lib', dylib_path]
        r = subprocess.run(cmd, capture_output=True, text=True, check=True)
        if r.returncode == 0:  print(f'✓ Applied usearch fix: Added /usr/lib rpath to {dylib_path}')
        else: print(f'✗ Failed to apply fix: {r.stderr}')
        _usearch_fix_applied = True
    except ImportError as ie:
        print('Warning: usearch not installed or import failed. you might need to install libsqlite3-dev. '
              'For macs do `brew install libsqlite3-dev`. For linux `apt install libsqlite3-dev`. ', ie)
    except subprocess.CalledProcessError as e: 
        # rpath already exists is not an error
        if 'duplicate' in str(e.stderr):
            _usearch_fix_applied = True
            return
        print(f'✗ install_name_tool failed: {e}')
        print(f'Command output: {e.output}')
        print(f'Command stderr: {e.stderr}')
    except Exception as e: print(f'Unexpected error during fix: {e}')

In [ ]:
#| hide
usearch_fix()

Applying usearch macOS fix if required...
usearch dylib path:  /Users/71293/code/litesearch/.venv/bin/usearch_binaries/usearch_sqlite.dylib
✗ install_name_tool failed: Command '['install_name_tool', '-add_rpath', '/usr/lib', '/Users/71293/code/litesearch/.venv/bin/usearch_binaries/usearch_sqlite.dylib']' returned non-zero exit status 1.
Command output: 
Command stderr: error: /Library/Developer/CommandLineTools/usr/bin/install_name_tool: for: /Users/71293/code/litesearch/.venv/bin/usearch_binaries/usearch_sqlite.dylib (for architecture arm64) option "-add_rpath /usr/lib" would duplicate path, file already has LC_RPATH for: /usr/lib

